In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import logging
from time import sleep

In [3]:
# Set up logging
logging.basicConfig(
                    level=logging.INFO,
                    format="%(asctime)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    )

In [4]:
# Load link and find all the names of categories and their links
url = 'https://www.kamazik.ru/zp/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
cats = soup.find('div', class_='col-lg-3 col-md-4 col-sm-4').findAll('li')        

In [5]:
cats

[<li><a href="/zp/gruppa_aksessuary/"><span></span>АКСЕССУАРЫ</a></li>,
 <li><a href="/zp/gruppa_gsm_smazki_i_zhidkosti/"><span></span>ГСМ (смазки и жидкости)</a></li>,
 <li><a href="/zp/gruppa_katalogi_i_rukovodstvo/"><span></span>КАТАЛОГИ И РУКОВОДСТВО</a></li>,
 <li><a href="/zp/gruppa_10_dvigatel/"><span>10</span>ДВИГАТЕЛЬ</a></li>,
 <li><a href="/zp/gruppa_11_sistema_pitaniya/"><span>11</span>СИСТЕМА ПИТАНИЯ</a></li>,
 <li><a href="/zp/gruppa_12_sistema_vypuska_gaza/"><span>12</span>СИСТЕМА ВЫПУСКА ГАЗА</a></li>,
 <li><a href="/zp/gruppa_13_sistema_okhlazhdeniya/"><span>13</span>СИСТЕМА ОХЛАЖДЕНИЯ</a></li>,
 <li><a href="/zp/gruppa_16_stseplenie/"><span>16</span>СЦЕПЛЕНИЕ</a></li>,
 <li><a href="/zp/gruppa_17_korobka_peredach/"><span>17</span>КОРОБКА ПЕРЕДАЧ</a></li>,
 <li><a href="/zp/gruppa_18_korobka_razdatochnaya/"><span>18</span>КОРОБКА РАЗДАТОЧНАЯ</a></li>,
 <li><a href="/zp/gruppa_22_valy_kardannye/"><span>22</span>ВАЛЫ КАРДАННЫЕ</a></li>,
 <li><a href="/zp/gruppa_23_most_p

In [6]:
# Get the categories names
cat_names = ([a.get_text(strip=False).split(maxsplit=0)[-1] 
              for a in soup.select('ul.catalog li a')]
            )
cat_names = [re.sub(r'^(\d+)', r'\1 ', cat) for cat in cat_names]
print(cat_names)

['АКСЕССУАРЫ', 'ГСМ (смазки и жидкости)', 'КАТАЛОГИ И РУКОВОДСТВО', '10 ДВИГАТЕЛЬ', '11 СИСТЕМА ПИТАНИЯ', '12 СИСТЕМА ВЫПУСКА ГАЗА', '13 СИСТЕМА ОХЛАЖДЕНИЯ', '16 СЦЕПЛЕНИЕ', '17 КОРОБКА ПЕРЕДАЧ', '18 КОРОБКА РАЗДАТОЧНАЯ', '22 ВАЛЫ КАРДАННЫЕ', '23 МОСТ ПЕРЕДНИЙ', '24 МОСТ ЗАДНИЙ', '25 МОСТ СРЕДНИЙ', '27 УСТРОЙСТВО СЕДЕЛЬНОЕ', '28 РАМА', '29 ПОДВЕСКА', '30 ОСЬ ПЕРЕДНЯЯ', '31 КОЛЕСА И СТУПИЦЫ', '34 УПРАВЛЕНИЕ РУЛЕВОЕ', '35 СИСТЕМА ТОРМОЗНАЯ', '37 ЭЛЕКТРООБОРУДОВАНИЕ', '38 ПРИБОРЫ', '39 ВОДИТЕЛЬСКИЙ ИНСТРУМЕНТ', '50 -57КАБИНА', '61 ДВЕРЬ КАБИНЫ', '68 СИДЕНЬЕ ВОДИТЕЛЯ', '80 ОБТЕКАТЕЛИ И СПОЙЛЕРЫ', '81 ОТОПЛЕНИЕ И ВЕНТИЛЯЦИЯ', '82 ПРИНАДЛЕЖНОСТИ КАБИНЫ', '84 ОПЕРЕНИЕ', '85 ПЛАТФОРМА', '86 МЕХАНИЗМ ПОДЪЕМА ПЛАТФОРМЫ']


In [7]:
# Get the categories links
cat_links = [link.find('a').get('href') for link in cats]
cat_links[:5]

['/zp/gruppa_aksessuary/',
 '/zp/gruppa_gsm_smazki_i_zhidkosti/',
 '/zp/gruppa_katalogi_i_rukovodstvo/',
 '/zp/gruppa_10_dvigatel/',
 '/zp/gruppa_11_sistema_pitaniya/']

In [8]:
# Parse all the necessary information
data = []

# Loop through links and category names 
for l, cat_name in zip(cat_links, cat_names):
    # Use f-string to construct all the categories links
    page_url = f'https://www.kamazik.ru{l}'
    r_2 = requests.get(page_url)
    soup_2 = BeautifulSoup(r_2.text, 'lxml')
    
    # Log category and page URL
    logging.info(f"Processing category: {cat_name}, Page URL: {page_url}")
    
    # Loop through second 2 pages of each category
    for p in range(1, 3):
        # Use f-string to construct for each category a pagination
        item_url = f'{page_url}?PAGEN_1={p}'
        r_3 = requests.get(item_url)
        sleep(1)
        soup_3 = BeautifulSoup(r_3.text, 'lxml')
        items = soup_3.findAll('div', class_='card')
        
        # Log last page of item URL
        if p == range(1, 3)[-1]:
            logging.info(f"Processing last pagination: {item_url}")
        
        # Use list comprehension to extract information for each category and items
        # and add it to data list
        data.extend(
            [
                [
                    cat_name,
                    item.find('div', class_='name').find('a').text.strip(),
                    item.find('div', class_='artikul').text.strip().replace('артикул', '').strip(),
                    item.find('div', class_='price').text.strip().replace('цена', '').strip(),
                ]
                for item in items
            ]
        )   

2024-12-22 20:11:37 - Processing category: АКСЕССУАРЫ, Page URL: https://www.kamazik.ru/zp/gruppa_aksessuary/
2024-12-22 20:11:40 - Processing last pagination: https://www.kamazik.ru/zp/gruppa_aksessuary/?PAGEN_1=2
2024-12-22 20:11:41 - Processing category: ГСМ (смазки и жидкости), Page URL: https://www.kamazik.ru/zp/gruppa_gsm_smazki_i_zhidkosti/
2024-12-22 20:11:44 - Processing last pagination: https://www.kamazik.ru/zp/gruppa_gsm_smazki_i_zhidkosti/?PAGEN_1=2
2024-12-22 20:11:45 - Processing category: КАТАЛОГИ И РУКОВОДСТВО, Page URL: https://www.kamazik.ru/zp/gruppa_katalogi_i_rukovodstvo/
2024-12-22 20:11:48 - Processing last pagination: https://www.kamazik.ru/zp/gruppa_katalogi_i_rukovodstvo/?PAGEN_1=2
2024-12-22 20:11:49 - Processing category: 10 ДВИГАТЕЛЬ, Page URL: https://www.kamazik.ru/zp/gruppa_10_dvigatel/
2024-12-22 20:11:52 - Processing last pagination: https://www.kamazik.ru/zp/gruppa_10_dvigatel/?PAGEN_1=2
2024-12-22 20:11:52 - Processing category: 11 СИСТЕМА ПИТАНИЯ, 

In [9]:
len(data)

1126

In [10]:
# Build a dataframe
df = pd.DataFrame(data, columns=['category', 'title', 'part_number', 'price'])
df.head(5)

,category,title,part_number,price
0,АКСЕССУАРЫ,Футболка FIRST KAMAZ 54901 черная (56 р-р),333.04180600,3 479 руб.
1,АКСЕССУАРЫ,Футболка FIRST KAMAZ 54901 черная (54 р-р),333.04180600,2 790 руб.
2,АКСЕССУАРЫ,Футболка FIRST KAMAZ 54901 черная (52 р-р),333.04180600,2 441 руб.
3,АКСЕССУАРЫ,Футболка FIRST KAMAZ 54901 белая (56 р-р),333.04180200,3 479 руб.
4,АКСЕССУАРЫ,Футболка FIRST KAMAZ 54901 белая (54 р-р),333.04180200,3 479 руб.


In [11]:
len(df)

1126

In [12]:
# Check data
df[25:40]

,category,title,part_number,price
25,АКСЕССУАРЫ,"Кабель для зарядки 3 в 1 (Type C, MicroUSB, iP...",20354,
26,АКСЕССУАРЫ,Дефлектор (большой угол) КамАЗ-5490,20859,
27,АКСЕССУАРЫ,Масштабная модель 4326 КамАЗ-МАСТЕР № 505,333.10050102,
28,АКСЕССУАРЫ,Кепка (цвет синий) КамАЗ-МАСТЕР,333.03050100,
29,АКСЕССУАРЫ,Масштабная модель KAMAZ -5490 белый эксклюзив,777.SSM1294,8 148 руб.
30,АКСЕССУАРЫ,Масштабная модель МКМ-4503-43253 мусоровоз,777.101975,6 006 руб.
31,АКСЕССУАРЫ,Игрушка-масштабная модель КАМАЗ-65206,777.101951,3 019 руб.
32,АКСЕССУАРЫ,Игрушка-масштабная модель КАМАЗ-5490,777.101944,4 891 руб.
33,АКСЕССУАРЫ,Игрушка-масштабная модель КАМАЗ-43502,777.101630,4 551 руб.
34,АКСЕССУАРЫ,"Масштабная модель 4326 ""КАМАЗ-Мастер"" 505",333.10050102,5 492 руб.
